In [1]:
!pip install beautifulsoup4 requests
!pip install schedule

In [2]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import schedule
import pandas as pd
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import spacy
import re


In [3]:
n_top_words = 50 
pagesToGet= 1
upperframe=[]  
submeu_li = []

nlp=spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

In [4]:
def get_soup(url1):

  page=""
  #an exception might be thrown, so the code should be in a try-except block
  try:
      #use the browser to get the url. This is suspicious command that might blow up.
      page=requests.get(url1)                             # this might throw an exception if something goes wrong.

  except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url1)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
  
  soup=BeautifulSoup(page.text,'html.parser')
  return soup

In [5]:
def get_news(): 
  url = "https://www.npr.org/sections/news/" 
  frame =[]
  soup = get_soup(url)
  links=soup.find_all(class_="item")
  
  for j in links:
    St = j.find("div",attrs={'class':'item-info'}).find("h2").text.strip()
    La = j.find("p").text.strip()
    if(len(La) > 20 and len(St) > 20):
      frame.append((St + " " +La))
    
  submeu=soup.find(class_="animated")#.find_all("li")

  for a in submeu.find_all('a', href=True):
    url = "https://www.npr.org" + a['href']
    soup = get_soup(url)
    links=soup.find_all(class_="item")
    for j in links:
      Statement = j.find("div",attrs={'class':'item-info'}).find("h2").text.strip()
      Label = j.find("p").text.strip()
      if(len(Label) > 20 and len(Statement) > 20):
        frame.append((Statement+ " " +Label))

  frame = [re.sub('\n','', f) for f in frame]
  frame = [re.sub(' +', ' ', f) for f in frame]
  frame = list(set(frame))
  return frame

In [6]:
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in all_stopwords]
    return (" ").join(tokens_filtered)

In [7]:
def data_cleaning(frame):
  lemma_frame_ls = []
  frame = [f.lower() for f in frame]
  frame = [re.sub('hide caption','', word) for word in frame]
  frame = [re.sub('getty images','', word) for word in frame]
  frame = [re.sub('[^A-Za-z ]+', '', word) for word in frame]

  frame =[remove_mystopwords(text) for text in frame]
  frame = ' '.join(frame)
  doc=nlp(frame)
  # Lemmatizing the text
  lemma_frame = [token.lemma_ for token in doc]
  lemma_frame_str = ' '.join(lemma_frame)

  lemma_frame_ls.append(lemma_frame_str)
  frame = [re.sub(' +', ' ', word) for word in lemma_frame_ls]
  return frame

In [8]:
def impl_lda():
  from datetime import datetime
  now = datetime.now()
  current_time = now.strftime("%H_%M_%S")
  print("Current thread execution time =", current_time)

  frame = data_cleaning(get_news())
  cv = CountVectorizer(stop_words='english', ngram_range=(3, 3))
  cvz = cv.fit_transform(frame)
  vocab = cv.get_feature_names_out()

  lda = LatentDirichletAllocation(n_components=1, max_iter = 5000, random_state=20)
  X_topics = lda.fit_transform(cvz)
  topic_words = lda.components_

  for i, topic_dist in enumerate(topic_words):
      sorted_topic_dist = np.argsort(topic_dist)
      topic_words = np.array(vocab)[sorted_topic_dist]
      topic_words = topic_words[:-n_top_words:-1]
      #print ("Topic", str(i+1), topic_words)

  print("Current Topic: ", topic_words)
  df = pd.DataFrame(topic_words)
  df.to_csv(f"a_{current_time}.csv", index = False, header=True)

In [9]:
def schedule_actions():
  schedule.every().day.at('12:00:00').do(impl_lda)
  #schedule.every(2).minutes.do(impl_lda)
  while True:
    schedule.run_pending()
    sleep(1)

schedule_actions()

Current thread execution time = 08_03_41
Current Topic:  ['new york city' 'incandescent light bulb' 'russias invasion ukraine'
 'tesla share sink' 'southern california resident'
 'marjorie taylor greene' 'leader kevin mccarthy' 'tesla ceo elon'
 'minority leader kevin' 'change climate change' 'house minority leader'
 'demby summer thomad' 'great barrier reef' 'taylor greene testify'
 'russiaukraine war happen' 'russian president vladimir'
 'intercept russian military' 'story air atc' 'ceo elon musk'
 'april rally scott' 'new movement stand' 'trump speak rally'
 'president donald trump' 'ice shelf size' 'climate change climate'
 'illegal voter registration' 'endorsement trump april'
 'climate scientist say' 'gene demby summer' 'thomad leah donnella'
 'americans live area' 'kevin mccarthy rcalif' 'elon musk speak'
 'ohio draw angerer' 'rally scott olson' 'karen grigsby bates'
 'trump april rally' 'delaware ohio draw' 'shelf size new'
 'war happen today' 'size new york' 'movement stand ro

KeyboardInterrupt: ignored